# KML data manipulation process

This is where we show our process of manipulating the KML data to get the desired output. In the project notebook, we'll be accessing these functions from `src/kml.py`. Below are the step to process the file:
- First, I parse the KML file and extract building footprints
- Define bounding box and create grids within the bounding box
- Define the functions to calculate building coverage in each grid
- Find the grid where the corresponding longitude and latitude belong to and assign the values of building coverage and building count to the original dataset

In [1]:
import os
from fastkml import kml
from lxml import etree
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import Polygon, box
from xml.dom import minidom


/Users/vandinh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Get the absolute path of the project's root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Change working directory
os.chdir(project_root)

In [3]:
# Path to the KML file
file_path = "data/raw/Building_Footprint.kml"

# Read the KML file as bytes
with open(file_path, "rb") as file:
    kml_data = file.read()

# Decode and remove XML declaration (to inspect the raw XML)
kml_str = kml_data.decode("utf-8").split("?>", 1)[-1].strip()

# Parse KML string using lxml to handle namespaces properly
root = etree.fromstring(kml_str)

# Use the correct namespace
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract the Folder and Placemark elements
folders = root.findall('.//kml:Folder', namespace)

# Check if we found folders and placemarks
if not folders:
    print("⚠️ No folders found in the KML file.")
else:
    print("Folders Found:")
    for folder in folders:
        # Safely extract the folder name
        folder_name = folder.find('kml:name', namespace)
        if folder_name is not None:
            print(f"  Folder Name: {folder_name.text}")
        else:
            print("  Folder Name: (No name provided)")

        # # Extract placemarks within this folder
        # for placemark in folder.findall('kml:Placemark', namespace):
        #     # Safely extract the placemark name
        #     placemark_name = placemark.find('kml:name', namespace)
        #     if placemark_name is not None:
        #         placemark_label = placemark_name.text
        #     else:
        #         placemark_label = "(No name provided)"

        #     # Print placemark name
        #     print(f"    Placemark Name: {placemark_label}")
            
        #     # Extract coordinates or other geometry
        #     coordinates = placemark.findall('.//kml:coordinates', namespace)
        #     if coordinates:
        #         print("      Coordinates:")
        #         for coord in coordinates:
        #             # Format the coordinates
        #             coord_list = coord.text.strip().split()
        #             for coordinate in coord_list:
        #                 lat, lon = coordinate.split(',')
        #                 print(f"        Latitude: {lat}, Longitude: {lon}")
        #     else:
        #         print("      Coordinates: (No coordinates found)")


Folders Found:
  Folder Name: Challenge_footprint


# **Parse the KML file and extract building footprints**


In [4]:
def parse_kml(file_path):
    """
    Description: Parse a KML file and extract building footprints as polygons.
    Handles both two and three component coordinates.
    Parameter (str): file_path: The path to KML file
    return: polygon (list).
    """
    kml = minidom.parse(file_path)
    polygons = []
    
    # Loop through each Placemark in the KML file
    for placemark in kml.getElementsByTagName("Placemark"):
        coords_text = placemark.getElementsByTagName("coordinates")[0].firstChild.nodeValue.strip()
        
        coords = []
        for coord in coords_text.split():
            # Split each coordinate into components
            coord_parts = coord.split(',')
            if len(coord_parts) == 2:
                lon, lat = map(float, coord_parts)  # Only longitude and latitude
            elif len(coord_parts) == 3:
                lon, lat, _ = map(float, coord_parts)  # Ignore altitude
            else:
                continue  # Skip invalid coordinates

            coords.append((lon, lat))
        
        polygons.append(Polygon(coords))  # Create a Polygon for each footprint

    return polygons

# Parse the KML file
polygons = parse_kml(file_path)
print(f"Extracted {len(polygons)} building footprints.")


Extracted 9436 building footprints.


# **Calculate Building Coverage**

**Define bounding box**

In [5]:
def extract_bounding_box(kml_file):
    tree = etree.parse(kml_file)
    root = tree.getroot()
    """
    Extracts the bounding box (minimum and maximum latitude and longitude) 
    from a KML file.

    This function:
    1. Parses the KML file and retrieves all coordinate elements.
    2. Iterates through the coordinates to determine the min/max latitude and longitude.
    3. Ignores altitude values if present.
    4. Returns the bounding box as (min_lat, max_lat, min_lon, max_lon).

    Parameters:
    kml_file (str): The file path to the KML file.

    Returns:
    tuple: (min_lat, max_lat, min_lon, max_lon) representing the bounding box.

    """

    # Define namespaces
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Find all coordinates within the KML file
    coords = root.xpath('.//kml:coordinates', namespaces=ns)

    min_lat, max_lat = float('inf'), float('-inf')
    min_lon, max_lon = float('inf'), float('-inf')

    # Iterate through all coordinates and find the bounding box
    for coord in coords:
        coords_text = coord.text.strip()
        for coord_pair in coords_text.split():
            coord_values = coord_pair.split(',')
            lon = float(coord_values[0])
            lat = float(coord_values[1])

            # Ignore the altitude (if present)
            if len(coord_values) == 3:
                _ = coord_values[2]  # We don't need altitude

            # Update the bounding box
            min_lat = min(min_lat, lat)
            max_lat = max(max_lat, lat)
            min_lon = min(min_lon, lon)
            max_lon = max(max_lon, lon)

    return min_lat, max_lat, min_lon, max_lon

# Let's use the data
min_lat, max_lat, min_lon, max_lon = extract_bounding_box(file_path)
print(f"Bounding Box: \nMin Lat: {min_lat}\nMax Lat: {max_lat}\nMin Lon: {min_lon}\nMax Lon: {max_lon}")


Bounding Box: 
Min Lat: 40.751285
Max Lat: 40.869321
Min Lon: -74.0022894813697
Max Lon: -73.869205


**Create grids within the bounding box**

In [6]:
#Define a function to create a grid of cells within the bounding box
def create_grid(min_lat, max_lat, min_lon, max_lon, cell_size=500):
    """
    Description: Creates a grid of rectangular cells within a given bounding box.

    This function:
    1. Converts latitude and longitude degrees into approximate meters.
    2. Determines the number of grid cells required in both latitude and longitude directions.
    3. Iterates through the bounding box to generate individual grid cells.
    4. Uses the `shapely.geometry.box` function to create rectangular polygons representing each grid cell.
    
    Parameters:
    min_lat (float): Minimum latitude of the bounding box.
    max_lat (float): Maximum latitude of the bounding box.
    min_lon (float): Minimum longitude of the bounding box.
    max_lon (float): Maximum longitude of the bounding box.
    cell_size (int, optional): Size of each grid cell in meters (default is 500m).

    Returns:
    list: A list of `shapely.geometry.Polygon` objects representing the grid cells.

    """
    # Convert degrees to meters (approximation at mid-latitude)
    lat_to_meters = 111320  # meters per degree of latitude
    lon_to_meters = 111320 * np.cos(np.radians((max_lat + min_lat) / 2))  # meters per degree of longitude at mid-latitude
    
    # Number of grid cells in each direction
    n_lat_cells = int((max_lat - min_lat) * lat_to_meters / cell_size)
    n_lon_cells = int((max_lon - min_lon) * lon_to_meters / cell_size)
    
    # Create the grid
    grid_cells = []
    for i in range(n_lat_cells):
        for j in range(n_lon_cells):
            # Calculate the bounds of each grid cell
            cell_min_lat = min_lat + i * cell_size / lat_to_meters
            cell_max_lat = min_lat + (i + 1) * cell_size / lat_to_meters
            cell_min_lon = min_lon + j * cell_size / lon_to_meters
            cell_max_lon = min_lon + (j + 1) * cell_size / lon_to_meters
            
            # Create a polygon for the grid cell
            grid_cell = box(cell_min_lon, cell_min_lat, cell_max_lon, cell_max_lat)
            grid_cells.append(grid_cell)
    
    return grid_cells

In [7]:
# Function to calculate building coverage in each grid cell
def calculate_coverage(building_footprints, grid_cells):
    coverage = []
    """
   Description: Calculates the total building coverage area within each grid cell. This function
    iterates over each grid cell in the provided list, checks for intersections between the grid cell and building footprints,
    computes the intersection area where buildings overlap with the grid cell, and then aggregates the total building coverage per grid cell.

    Parameters:
    building_footprints: A list of polygons representing building footprints.
    grid_cells (list of shapely.geometry.Polygon): A list of polygons representing the grid cells.

    Returns:
    list: A list of coverage values where each entry corresponds to the total building area within a grid cell.
    """
    
    # Loop over each grid cell and calculate the intersection with building footprints
    for cell in grid_cells:
        cell_coverage = 0
        for building in building_footprints:
            if building.intersects(cell):
                # Calculate the intersection area between the building and the grid cell
                intersection = building.intersection(cell)
                cell_coverage += intersection.area
        
        # Store the coverage for this grid cell
        coverage.append(cell_coverage)
    
    return coverage

In [8]:
# Function to compute building count per grid cell without buffering
def building_count(building_footprints, grid_cells):
    building_counts = []
    """
   Description: Computes the number of buildings that intersect each grid cell.This function iterates over a list of grid cells and counts how many building footprints
    intersect each grid cell without applying any buffering.

    Parameters:
    building_footprints (list of shapely.geometry.Polygon): A list of polygons representing building footprints.
    grid_cells (list of shapely.geometry.Polygon): A list of polygons representing the grid cells.

    Returns:
    list: A list where each entry corresponds to the number of buildings intersecting a grid cell.
    """
    
    # Loop through grid cells to count how many buildings intersect each grid cell
    for cell in grid_cells:
        count = sum(1 for building in building_footprints if building.intersects(cell))  # Direct intersection without buffer
        building_counts.append(count)
    
    return building_counts

In [9]:
from shapely.geometry import Polygon, box
# Create the grid cells within the bounding box
grid_cells = create_grid(min_lat, max_lat, min_lon, max_lon, cell_size=500)

# Calculate the building coverage for each grid cell
coverage = calculate_coverage(polygons, grid_cells)
building_counts = building_count(polygons, grid_cells)

In [10]:
# Create DataFrame to hold the results
grid_cell_ids = [f"Grid Cell {i+1}" for i in range(len(grid_cells))]  # Grid cell IDs
coverage_df = pd.DataFrame({
    "Grid Cell ID": grid_cell_ids,
    "Coverage (squaremeters)": coverage,
    "Building Count": building_counts
})

# Display the DataFrame
print(coverage_df.head())

  Grid Cell ID  Coverage (squaremeters)  Building Count
0  Grid Cell 1             0.000000e+00               0
1  Grid Cell 2             0.000000e+00               0
2  Grid Cell 3             0.000000e+00               0
3  Grid Cell 4             0.000000e+00               0
4  Grid Cell 5             5.615163e-07               5


**Percentage format**

In [11]:
# Step 1: Function to compute building coverage percentage
def calculate_coverage_percentage(building_footprints, grid_cells):
    coverage = []
    coverage_pct = []

    # Compute total area of a single grid cell
    cell_area = grid_cells[0].area  # Assuming all grid cells have the same size

    for cell in grid_cells:
        cell_coverage = 0  # Initialize coverage per cell

        for building in building_footprints:
            if building.intersects(cell):
                intersection = building.intersection(cell)
                cell_coverage += intersection.area  # Sum up intersection areas

        # Compute coverage percentage
        coverage.append(cell_coverage)
        coverage_pct.append((cell_coverage / cell_area) * 100 if cell_area > 0 else 0)

    return coverage, coverage_pct

# Step 2: Compute coverage metrics
coverage, coverage_pct = calculate_coverage_percentage(polygons, grid_cells)

# Step 3: Create DataFrame
grid_cell_ids = [f"Grid Cell {i+1}" for i in range(len(grid_cells))]

coverage_df = pd.DataFrame({
    "Grid Cell ID": grid_cell_ids,
    "Coverage (sq meters)": coverage,
    "Coverage (%)": coverage_pct,
    "Building Count": building_counts
})

# Display first rows
print(coverage_df.head())


  Grid Cell ID  Coverage (sq meters)  Coverage (%)  Building Count
0  Grid Cell 1          0.000000e+00       0.00000               0
1  Grid Cell 2          0.000000e+00       0.00000               0
2  Grid Cell 3          0.000000e+00       0.00000               0
3  Grid Cell 4          0.000000e+00       0.00000               0
4  Grid Cell 5          5.615163e-07       2.10666               5


In [16]:
coverage_df.describe()

,Coverage (sq meters),Coverage (%),Building Count
count,5.720000e+02,572.000000,572.000000
mean,3.490643e-06,13.095963,19.648601
std,4.570108e-06,17.145829,28.447585
min,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000,0.000000
50%,2.455443e-07,0.921217,3.000000
75%,7.075855e-06,26.546723,32.000000
max,1.634728e-05,61.330630,191.000000


# **Assign values to the original dataset**

In [12]:
# Step 1: Load the original dataset
train_df = pd.read_csv("data/raw/Training_data_uhi_index_2025-02-18.csv")
test_df = pd.read_csv("data/test/Submission_template.csv")

In [13]:
def assign_coverage_and_count_to_points(train_df, grid_cells, coverage, building_counts):
    assigned_coverage = []
    assigned_building_count = []
    
    for _, row in train_df.iterrows():
        point = Point(row['Longitude'], row['Latitude'])  # Create a point from longitude and latitude
        
        # Default values
        point_coverage = 0  # Default coverage is 0 if the point is not in any grid cell
        point_building_count = 0  # Default building count is 0
        
        # Check which grid cell the point belongs to
        for i, grid_cell in enumerate(grid_cells):
            if grid_cell.contains(point):
                point_coverage = coverage[i]  # Assign coverage from the grid cell
                point_building_count = building_counts[i]  # Assign building count from the grid cell
                break  # Stop checking further once the point is found in a grid cell
        
        assigned_coverage.append(point_coverage)
        assigned_building_count.append(point_building_count)
    
    # Assign the calculated coverage and building count to the dataframe
    train_df['building_coverage'] = assigned_coverage
    train_df['building_count'] = assigned_building_count
    return train_df

# Step 3: Apply the function to your dataset
ground_df = assign_coverage_and_count_to_points(train_df, grid_cells, coverage, building_counts)
test_ground_df = assign_coverage_and_count_to_points(test_df, grid_cells, coverage, building_counts)

# Step 4: Save the updated dataframe with the building coverage and count columns
ground_df.to_csv("data/interim/ground_df.csv", index=False)
test_ground_df.to_csv("data/interim/test_ground_df.csv", index=False)

# Preview the updated dataset
print(ground_df.head())
print(test_ground_df.head())

   Longitude   Latitude          datetime  UHI Index  building_coverage  \
0 -73.909167  40.813107  24-07-2021 15:53   1.030289           0.000006   
1 -73.909187  40.813045  24-07-2021 15:53   1.030289           0.000006   
2 -73.909215  40.812978  24-07-2021 15:53   1.023798           0.000006   
3 -73.909242  40.812908  24-07-2021 15:53   1.023798           0.000006   
4 -73.909257  40.812845  24-07-2021 15:53   1.021634           0.000006   

   building_count  
0              44  
1              44  
2              44  
3              44  
4              44  
   Longitude   Latitude  UHI Index  building_coverage  building_count
0 -73.971665  40.788763        NaN           0.000012              43
1 -73.971928  40.788875        NaN           0.000012              43
2 -73.967080  40.789080        NaN           0.000012              43
3 -73.972550  40.789082        NaN           0.000012              43
4 -73.969697  40.787953        NaN           0.000012              43


/Users/vandinh/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [14]:
ground_df.head()

,Longitude,Latitude,datetime,UHI Index,building_coverage,building_count
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,0.000006,44
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,0.000006,44
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,0.000006,44
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,0.000006,44
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,0.000006,44
